In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import clear_output
from torchviz import make_dot



In [2]:
SEQ_LENGTH = 1024  # Longueur des séries temporelles
LATENT_DIM = 64    # Dimension de l'espace latent
BATCH_SIZE = 32
EPOCHS = 30000
SEQS = 1000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
df = pd.read_pickle('assets/binance_BTC_USDT.zip')
size = df.shape[0]
deb_max = size - SEQ_LENGTH

arr=[]
for i in range(SEQS):
    deb = np.random.randint(0, deb_max)
    df2 = df.iloc[deb:deb+SEQ_LENGTH+1]
    # close = df2.close.array
    close = df2.close.pct_change().dropna().array
    close = 2 * (close - close.min()) / (close.max() - close.min()) - 1
    arr.append(close)

arr = np.array(arr)
data = arr.reshape((SEQS, 1, SEQ_LENGTH)).astype(np.float32)
# print(arr)
print(arr.shape)


(1000, 1024)
